# RPG-Würfel

In Rollenspielen werden Konflikte zwischen Spielern oft durch Würfeln
entschieden. Dabei werden oft mehrere Würfel gleichzeitig verwendet. Außerdem
werden nicht nur die bekannten 6-seitigen Würfel verwendet, sondern auch
4-seitige, 8-seitige, 20-seitige Würfel, etc.

Die Anzahl und Art der Würfel wird dabei durch folgende Notation beschrieben:

```text
<Anzahl der Würfel> d <Seiten pro Würfel>
```

Zum Beispiel wird das Würfeln mit zwei 6-seitigen Würfeln als `2d6`
beschrieben. Manchmal werden auch komplexere Formeln verwendet: 
`3d20 + 2d6 - 4` bedeutet, dass gleichzeitig drei 20-seitige Würfel und zwei 6-seitige
Würfel geworfen werden und die Gesamtsumme der Augenzahlen dann um 4
verringert wird.

In manchen Spielen wird das Werfen der niedrigsten oder höchsten Augenzahl
besonders behandelt ("katastrophale Niederlage", "kritischer Erfolg").

In den folgenden Aufgaben sollen Sie derartige RPG-Würfel in Python
implementieren. Um Ihre Implementierung testen zu können empfiehlt es sich
sie in einem IDE zu realisieren. 

Schreiben Sie Tests für jede Funktionalität, die Sie implementieren.
Wie können Sie beim Testen mit der Zufälligkeit beim Würfeln umgehen?
Was sind Stärken bzw. Schwächen der von Ihnen gewählten Teststrategie?

## Generalisierte Würfel: Interface `Dice`

Implementieren Sie eine abstrakte Klasse `Dice`, die das Würfeln mit
beliebigen Würfeln beschreibt: Die Klasse soll folgende abstrakte Methoden
anbieten:

- `roll(): int` liefert das Ergebnis eines Wurfes mit den entsprechenden
  Würfeln zurück.

- properties `max_value: int` und `min_value: int` geben den kleinsten bzw.
  größten Wert zurück, der mit den entsprechenden Würfeln geworfen werden
  kann.

Falls Sie Pytest verwenden:
Schreiben Sie parametrische Pytest Tests, mit denen (indirekte) Instanzen
von `Dice` getestet werden können.

In [4]:
import random
from abc import ABC, abstractmethod
from typing import Tuple, Callable, Sequence, Union, Iterable

In [5]:
class Dice(ABC):
    """Roll with a combination of multiple dice."""

    @abstractmethod
    def roll(self) -> int:
        ...

    @property
    @abstractmethod
    def min_value(self) -> int:
        ...

    @property
    @abstractmethod
    def max_value(self) -> int:
        ...

## Klasse `ConstantDice`

Implementieren Sie eine Klasse `ConstantDice`, die das Interface `Dice`
implementiert und einen Würfel realisiert, der immer einen konstanten, bei der
Instanziierung des Objekts festgelegten Wert würfelt.

In [6]:
from dataclasses import dataclass
@dataclass
class ConstantDice(Dice):
    value: int

    def roll(self) -> int:
        return self.value

    @property
    def min_value(self) -> int:
        return self.value

    @property
    def max_value(self) -> int:
        return self.value

In [7]:
dice = ConstantDice(3)
assert dice.min_value == 3
assert dice.max_value == 3
assert dice.roll() == 3

## Klasse `FairDice`

Implementieren Sie eine Klasse `FairDice`, die das Interface `Dice`
implementiert und einen der oben beschriebenen Würfe mit mehreren Würfeln
einer beliebigen (aber für alle Würfel gleichen) Augenzahl realisiert. Z.B.
sollen Würfe der Form `3d6` oder `4d17` durch Instanzen dieser Klasse
darstellbar sein (obwohl es in der Realität keinen 17-seitigen (fairen)
Würfel gibt).

Wie können Sie beim Testen mit der Zufälligkeit beim Würfeln umgehen? Was
sind Stärken bzw. Schwächen der gewählten Teststrategie?

*Hinweis:* Die Funktionen `random.randint()` und `random.seed()` können 
bei dieser Aufgabe hilfreich sein.

In [ ]:
class FairDice(Dice):
    def __init__(self, num_dice, num_sides):
        assert num_dice >= 1
        assert num_sides >= 2

        self.num_dice = num_dice
        self.num_sides = num_sides

    def __eq__(self, other):
        if isinstance(other, FairDice):
            return (self.num_dice == other.num_dice
                    and self.num_sides == other.num_sides)
        else:
            return False

    def roll(self) -> int:
        result = 0
        for _ in range(self.num_dice):
            result += random.randint(1, self.num_sides)
        return result

    @property
    def min_value(self) -> int:
        return self.num_dice

    @property
    def max_value(self) -> int:
        return self.num_sides * self.num_dice

In [ ]:
dice = FairDice(2, 6)
assert dice.min_value == 2
assert dice.max_value == 12
random.seed(1)
assert dice.roll() == 7

## Klasse `SumDice`

Implementieren Sie eine Klasse `SumDice`, die das Interface `Dice`
implementiert und die Summe des Würfelns mit mehreren verschiedenen Würfeln
(potentiell der Form `<m>d<n>`) repräsentiert.

In [ ]:
class SumDice(Dice):
    def __init__(self, dice: Iterable):
        assert dice
        self.dice = dice

    def __eq__(self, other):
        if isinstance(other, SumDice):
            return self.dice == other.dice
        else:
            return False

    def roll(self) -> int:
        return sum(d.roll() for d in self.dice)

    @property
    def min_value(self) -> int:
        return sum(d.min_value for d in self.dice)

    @property
    def max_value(self) -> int:
        return sum(d.max_value for d in self.dice)



In [ ]:
dice = SumDice([ConstantDice(3), ConstantDice(2)])
assert dice.min_value == 5
assert dice.max_value == 5
assert dice.roll() == 5

In [ ]:
dice = SumDice([ConstantDice(1), FairDice(2, 6)])
assert dice.min_value == 3
assert dice.max_value == 13
random.seed(1)
assert dice.roll() == 8

## Klasse `SimpleDie`

Implementieren Sie eine Klasse `SimpleDie`, die das Interface `Dice`
implementiert und das Würfeln mit einem Würfel beliebiger, bei der
Instanziierung des Würfels festgelegten, Seitenzahl ermöglicht.

In [ ]:
class SimpleDie(Dice):
    def __init__(self, num_sides):
        assert num_sides >= 2
        self.num_sides = num_sides

    def __eq__(self, other):
        if isinstance(other, SimpleDie):
            return self.num_sides == other.num_sides
        else:
            return False

    def roll(self) -> int:
        return random.randint(1, self.num_sides)

    @property
    def min_value(self) -> int:
        return 1

    @property
    def max_value(self) -> int:
        return self.num_sides

In [ ]:
die = SimpleDie(6)
assert die.min_value == 1
assert die.max_value == 6
random.seed(1)
assert die.roll() == 2

## Klasse `MultipleRollDice`

Implementieren Sie eine Klasse `MultipleRollDice`, die das Interface `Dice`
implementiert und das wiederholte Würfeln mit einem (generalisierten)
Würfel realisiert.

Was ist die Beziehung zwischen `FairDice` und der Kombination aus
`SimpleDie` und `MultipleRollDice`? Wie unterscheiden sich die beiden
Implementierungsstrategien in ihrer Testbarkeit.

In [ ]:
class MultipleRollDice(Dice):
    def __init__(self, rolls, dice):
        assert rolls >= 1
        assert dice
        self.rolls = rolls
        self.dice = dice

    def __eq__(self, other):
        if isinstance(other, MultipleRollDice):
            return self.rolls == other.rolls and self.dice == other.dice
        else:
            return False

    def roll(self) -> int:
        result = 0
        for _ in range(self.rolls):
            result += self.dice.roll()
        return result

    @property
    def min_value(self) -> int:
        return self.rolls * self.dice.min_value

    @property
    def max_value(self) -> int:
        return self.rolls * self.dice.max_value



In [ ]:
dice = MultipleRollDice(2, SimpleDie(6))
assert dice.min_value == 2
assert dice.max_value == 12
random.seed(1)
assert dice.roll() == 7

# Factory für RPG-Würfel

Schreiben Sie eine Funktion `create_dice(configuration: str) -> Dice`,
die eine Konfiguration von RPG-Würfeln als Argument bekommt und eine
geeignete Konfiguration von Dice-Instanzen zurückgibt.  Zum Beispiel soll
für "3d8 + 6" ein `SumDice` zurückgegeben werden, der einen `FairDice` (mit
3 8-seitigen Würfeln) und einen `ConstantDice` mit dem Wert 6 enthält.